<a href="https://colab.research.google.com/github/obedotto/monte-carlo-control/blob/main/MonteCarloControlExp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-lq8e8i2y/gym-walk_26cb1d80864b4ebf8984babf6144fa49
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-lq8e8i2y/gym-walk_26cb1d80864b4ebf8984babf6144fa49
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done


In [ ]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np
import tqdm as tqdm
np.bool8=np.bool_
import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [ ]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [ ]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [ ]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
#LEFT, RIGHT = range(2)

In [ ]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

Exponentially decaying schedule


In [ ]:
def decay_schedule(
    init_values, min_values, decay_ratio,
    max_steps, log_start = -2, log_base=10):
  decay_steps = int(max_steps*decay_ratio)
  rem_steps=max_steps - decay_steps
  values = np.logspace(log_start,0,decay_steps,base=log_base,endpoint=True)[::-1]
  values = (values - values.min())/(values.max() - values.min())
  values = (init_values - min_values)*values + min_values
  values = np.pad(values,(0,rem_steps),'edge')
  return values

Exploratory Policy Trajectories

In [ ]:
from itertools import count
def generate_trajectory(
    select_action, Q, epsilon,
    env, max_steps=200):
  done, trajectory = False, []
  while not done:
    state = env.reset()
    for t in count():
      action = select_action(state,Q,epsilon)
      next_state, reward, done, _ = env.step(action)
      experience = (state,action,reward,next_state,done)
      trajectory.append(experience)
      if done:
        break
      if t >= max_steps - 1:
        trajectory = []
        break
      state = next_state
  return np.array(trajectory, np.object)

Monte Carlo control

In [ ]:
def mc_control (env, gamma = 1.0,
                init_alpha = 0.5,min_alpha = 0.01, alpha_decay_ratio = 0.5,
                init_epsilon = 1.0, min_epsilon = 0.1, epsilon_decay_ratio = 0.9,
                n_episodes = 3000, max_steps = 200, first_visit = True):
  nS, nA = env.observation_space.n, env.action_space.n
  discounts = np.logspace(0,max_steps,num=max_steps,base=gamma,endpoint=False)
  alphas = decay_schedule(init_alpha,min_alpha,alpha_decay_ratio,n_episodes)
  epsilons = decay_schedule(init_epsilon,min_epsilon,epsilon_decay_ratio,n_episodes)
  pi_track = []
  Q =np.zeros((nS,nA),dtype=np.float64)
  Q_track = np.zeros((n_episodes,nS,nA),dtype=np.float64)
  select_action = lambda state,Q,epsilon:np.argmax(Q[state]) if np.random.random() > epsilon else np.random.randint(len(Q[state]))
  for e in tqdm.tqdm(range(n_episodes),leave=False):
    trajectory = generate_trajectory(select_action,Q,epsilons[e],env,max_steps)
    visited = np.zeros((nS,nA),dtype = np.bool)
    for t,(state,action,reward,_,_) in enumerate(trajectory):
      if visited[state][action] and first_visit:
        continue
      visited[state][action] = True
      n_steps = len(trajectory[t:])
      G = np.sum(discounts[:n_steps]*trajectory[t:,2])
      Q[state][action] = Q[state][action]+alphas[e]*(G - Q[state][action])
      Q_track[e] = Q
      pi_track.append(np.argmax(Q,axis=1))
  V = np.max(Q,axis=1)
  pi = lambda s: {s:a for s,a in enumerate(np.argmax(Q,axis=1))}[s]
  return Q,V,pi,Q_track,pi_track

In [ ]:
import numpy as np

if not hasattr(np, "object"):
  np.object = object

In [ ]:
optimal_Q, optimal_V, optimal_pi,_,_ = mc_control (env,n_episodes = 3000)
print('Name: Sandhiya R         Register Number:212223240146         ')
print_state_value_function(optimal_Q, P, n_cols=4, prec=2, title='Action-value function:')
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(optimal_pi, P)

Name: Sandhiya R         Register Number:212223240146         
Action-value function:
| 00 [0.36 0.34 0.32 0.33] | 01 [0.14 0.13 0.08 0.31] | 02 [0.12 0.25 0.19 0.16] | 03 [0.09 0.08 0.04 0.18] |
| 04 [0.37 0.23 0.21 0.25] |           | 06 [0.11 0.06 0.25 0.01] |           |
| 08 [0.26 0.3  0.22 0.42] | 09 [0.28 0.5  0.33 0.29] | 10 [0.48 0.17 0.26 0.2 ] |           |
|           | 13 [0.33 0.36 0.61 0.32] | 14 [0.4  0.8  0.64 0.55] |           |
State-value function:
| 00   0.36 | 01   0.31 | 02   0.25 | 03   0.18 |
| 04   0.37 |           | 06   0.25 |           |
| 08   0.42 | 09    0.5 | 10   0.48 |           |
|           | 13   0.61 | 14    0.8 |           |
Policy:
| 00      < | 01      ^ | 02      v | 03      ^ |
| 04      < |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [ ]:
# Find the probability of success and the mean return of you your policy
print('Name: Sandhiya R            Register Number: 212223240146')
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=goal_state)*100,
    mean_return(env, optimal_pi)))


Name: Sandhiya R            Register Number: 212223240146
Reaches goal 16.00%. Obtains an average undiscounted return of 0.1600.
